In [0]:
%sql
DROP DATABASE IF EXISTS legend CASCADE;
CREATE DATABASE legend;

## Legend model
We can load a Legend data model from classpath or directory as follows

In [0]:
%scala
import org.finos.legend.spark.LegendClasspathLoader
val legend = LegendClasspathLoader.loadResources()

import org.finos.legend.spark.LegendClasspathLoader
legend: org.finos.legend.spark.Legend = org.finos.legend.spark.Legend@5fc16956

## Legend entities
All available entities will be retrieved from our legend data model

In [0]:
%scala
val entities = legend.getEntityNames
display(entities.toSeq.toDF("pure"))

pure
databricks::runtime::employee
databricks::entity::person
databricks::entity::sme
databricks::service::skills
databricks::lakehouse::employee
databricks::entity::employee
databricks::table::employee
databricks::service::employee
databricks::mapping::employee_delta


## Legend schema
We can create the spark schema for any Legend entity. This process will recursively loop through each underlying fields, enums, nested properties and supertypes.

In [0]:
%scala
val schema = legend.getSchema("databricks::entity::employee")
display(schema.fields.map(s => s.toDDL).toSeq.toDF("field"))

field
firstName STRING NOT NULL COMMENT 'Person first name'
lastName STRING NOT NULL COMMENT 'Person last name'
birthDate DATE NOT NULL COMMENT 'Person birth date'
gender STRING COMMENT 'Person gender'
id INT COMMENT 'Unique ID of a databricks employee'
sme STRING COMMENT 'Programming skill that person truly masters'
joinedDate DATE NOT NULL COMMENT 'When did that person join Databricks'
highFives INT COMMENT 'How many high fives did that person get'


## Legend transformations
We can transform raw entities into their desired target tables. Note that relational transformations only support direct mapping and therefore easily enforced through `.withColumnRenamed` syntax.

In [0]:
%scala
val transformations = legend.getTransformations("databricks::mapping::employee_delta")
display(transformations.toSeq.toDF("column", "columnRenamed"))

column,columnRenamed
highFives,high_fives
joinedDate,joined_date
lastName,last_name
firstName,first_name
birthDate,birth_date
id,id
sme,sme
gender,gender


## Legend expectations
Given the `multiplicity` properties, we can 
detect if a field is optional or not or list has the right number of elements. Given an `enumeration`, 
we check for value consistency. These will be considered **technical expectations** and converted into SQL constraints. In addition, we also support the conversion of **business expectations**
from the PURE language to SQL expressions. We generate a legend
execution plan against a Databricks runtime

In [0]:
%scala
val expectations = legend.getExpectations("databricks::mapping::employee_delta")
display(expectations.toSeq.toDF("name", "expectation"))

name,expectation
[birthDate] is mandatory,birth_date IS NOT NULL
[sme] not allowed value,"(sme IS NULL OR sme IN ('Scala', 'Python', 'Java', 'R', 'SQL'))"
[joinedDate] is mandatory,joined_date IS NOT NULL
[firstName] is mandatory,first_name IS NOT NULL
[high five] should be positive,(high_fives IS NOT NULL AND high_fives > 0)
[lastName] is mandatory,last_name IS NOT NULL
[hiringAge] should be > 18,year(joined_date) - year(birth_date) > 18


## Legend derivations
We can convert Legend derived properties as SQL expressions. In the example model, the field `age` is not physically stored but can be computed at runtime.

In [0]:
%scala
val derivations = legend.getDerivations("databricks::mapping::employee_delta")
display(derivations.toSeq.toDF("column", "expression"))

column,expression
hiringAge,year(joined_date) - year(birth_date) AS `hiringAge`
age,year(current_date) - year(birth_date) AS `age`


## Legend tables
In order to query our validated entity from legend interface, we can easily create the target state table.

In [0]:
%scala
val tableName = legend.createTable("databricks::mapping::employee_delta")
display(sql(s"DESCRIBE EXTENDED $tableName"))

col_name,data_type,comment
first_name,string,Person first name
last_name,string,Person last name
birth_date,date,Person birth date
gender,string,Person gender
id,int,Unique ID of a databricks employee
sme,string,Programming skill that person truly masters
joined_date,date,When did that person join Databricks
high_fives,int,How many high fives did that person get
,,
# Partitioning,,


# Example - write
In this scenario, we read raw JSON files that we schematize, transform and persist to our target state delta table.

In [0]:
%sh
head /dbfs/FileStore/antoine.amend@databricks.com/legend/employee.json

{"birthDate":"1989-02-19","firstName":"Levey","gender":"M","highFives":282,"joinedDate":"2015-12-05","lastName":"Storck","role":"SA","sme":"C"}
{"birthDate":"1987-08-14","firstName":"Maria","gender":"M","highFives":299,"id":2,"joinedDate":"2017-03-03","lastName":"O'Gorman","role":"SA","sme":"Python"}
{"birthDate":"1970-10-04","firstName":"Evvy","gender":"M","highFives":182,"id":3,"joinedDate":"2020-11-02","lastName":"Lepoidevin","role":"GTM","sme":"C"}
{"birthDate":"1973-11-26","firstName":"Georges","gender":"F","highFives":229,"id":4,"joinedDate":"2020-09-14","lastName":"Jotcham","role":"GTM","sme":"Scala"}
{"birthDate":"1987-03-11","firstName":"Doroteya","gender":"N","highFives":78,"id":5,"joinedDate":"2019-02-11","lastName":"Wadhams","role":"ENG","sme":"Scala"}
{"birthDate":"1988-08-01","firstName":"Mia","gender":"F","highFives":146,"id":6,"joinedDate":"2017-04-13","lastName":"Millgate","role":"ENG","sme":"Python"}
{"birthDate":"1979-07-15","firstName":"Celene","gender":"N","highFives":69,"id":7,"joinedDate":"2021-06-03","lastName":"Calverley","role":"ENG","sme":"Python"}
{"birthDate":"1980-05-18","firstName":"Richie","gender":"F","highFives":167,"id":8,"joinedDate":"2014-08-23","lastName":"Di Matteo","role":"SA","sme":"Python"}
{"birthDate":"1987-01-10","firstName":"Ignaz","gender":"F","highFives":199,"joinedDate":"2014-02-01","lastName":"Kurth","role":"RSA","sme":"Python"}
{"birthDate":"1998-02-08","firstName":"Anthia","gender":"F","highFives":277,"id":10,"joinedDate":"2015-01-14","lastName":"Duck","role":"RSA","sme":"Python"}

In [0]:
%scala
val schema = legend.getSchema("databricks::entity::employee")
val schemaDf = spark.read.format("json").schema(schema).load("/FileStore/antoine.amend@databricks.com/legend")
display(schemaDf.limit(10))

firstName,lastName,birthDate,gender,id,sme,joinedDate,highFives
Levey,Storck,1989-02-19,M,null,C,2015-12-05,282
Maria,O'Gorman,1987-08-14,M,2,Python,2017-03-03,299
Evvy,Lepoidevin,1970-10-04,M,3,C,2020-11-02,182
Georges,Jotcham,1973-11-26,F,4,Scala,2020-09-14,229
Doroteya,Wadhams,1987-03-11,N,5,Scala,2019-02-11,78
Mia,Millgate,1988-08-01,F,6,Python,2017-04-13,146
Celene,Calverley,1979-07-15,N,7,Python,2021-06-03,69
Richie,Di Matteo,1980-05-18,F,8,Python,2014-08-23,167
Ignaz,Kurth,1987-01-10,F,null,Python,2014-02-01,199
Anthia,Duck,1998-02-08,F,10,Python,2015-01-14,277


In [0]:
%scala
import org.finos.legend.spark._
val transformations = legend.getTransformations("databricks::mapping::employee_delta")
val transformedDf = schemaDf.legendTransform(transformations)
display(transformedDf.limit(10))

first_name,last_name,birth_date,gender,id,sme,joined_date,high_fives
Levey,Storck,1989-02-19,M,null,C,2015-12-05,282
Maria,O'Gorman,1987-08-14,M,2,Python,2017-03-03,299
Evvy,Lepoidevin,1970-10-04,M,3,C,2020-11-02,182
Georges,Jotcham,1973-11-26,F,4,Scala,2020-09-14,229
Doroteya,Wadhams,1987-03-11,N,5,Scala,2019-02-11,78
Mia,Millgate,1988-08-01,F,6,Python,2017-04-13,146
Celene,Calverley,1979-07-15,N,7,Python,2021-06-03,69
Richie,Di Matteo,1980-05-18,F,8,Python,2014-08-23,167
Ignaz,Kurth,1987-01-10,F,null,Python,2014-02-01,199
Anthia,Duck,1998-02-08,F,10,Python,2015-01-14,277


In [0]:
%scala
val tableName = legend.getTable("databricks::mapping::employee_delta")
transformedDf.write.format("delta").mode("append").saveAsTable(tableName)

tableName: String = legend.employee

# Example - drift
Given our delta backend, we may decide to allow / decline data drift. A new schema update would result in seamless execution of pipeline with zero code overhead

In [0]:
%scala
// Let's create an additional field
import org.apache.spark.sql.functions._
val updated_schemaDf = spark.read.format("json").schema(schema).load("/FileStore/antoine.amend@databricks.com/legend").withColumn("isMvp", lit(true))
val updated_transformedDf = updated_schemaDf.legendTransform(transformations)
display(updated_transformedDf)

first_name,last_name,birth_date,gender,id,sme,joined_date,high_fives,isMvp
Levey,Storck,1989-02-19,M,null,C,2015-12-05,282,true
Maria,O'Gorman,1987-08-14,M,2,Python,2017-03-03,299,true
Evvy,Lepoidevin,1970-10-04,M,3,C,2020-11-02,182,true
Georges,Jotcham,1973-11-26,F,4,Scala,2020-09-14,229,true
Doroteya,Wadhams,1987-03-11,N,5,Scala,2019-02-11,78,true
Mia,Millgate,1988-08-01,F,6,Python,2017-04-13,146,true
Celene,Calverley,1979-07-15,N,7,Python,2021-06-03,69,true
Richie,Di Matteo,1980-05-18,F,8,Python,2014-08-23,167,true
Ignaz,Kurth,1987-01-10,F,null,Python,2014-02-01,199,true
Anthia,Duck,1998-02-08,F,10,Python,2015-01-14,277,true


In [0]:
%scala
try {
  updated_transformedDf.write.format("delta").mode("append").saveAsTable(tableName)
} catch {
  case e: Exception => println(e.getMessage())
}

A schema mismatch detected when writing to the Delta table (Table ID: ee04410d-53db-4f37-a977-8f0960ed5e78).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- birth_date: date (nullable = true)
-- gender: string (nullable = true)
-- id: integer (nullable = true)
-- sme: string (nullable = true)
-- joined_date: date (nullable = true)
-- high_fives: integer (nullable = true)


Data schema:
root
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- birth_date: date (nullable = true)
-- gender: string (nullable = true)
-- id: integer (nullable = true)
-- sme: string (nullable = true)
-- joined_date: date (nullable = true)
-- high_fives: integer (nullable = true)
-- isMvp: boolean (nullable = true)

In [0]:
%scala
updated_transformedDf.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(tableName)

In [0]:
%scala
val versionDF = sql(s"DESCRIBE HISTORY ${tableName}")
val versionLatest = versionDF.select(max("version").alias("version")).collect().head.getAs[Long]("version")
display(versionDF)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2022-07-16T05:11:42.800+0000,999271697022884,antoine.amend@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3591277798165548),0410-051949-9j5ul4vr,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 33274)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-07-16T05:10:32.686+0000,999271697022884,antoine.amend@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3591277798165548),0410-051949-9j5ul4vr,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 33051)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-07-16T05:09:59.186+0000,999271697022884,antoine.amend@databricks.com,CREATE TABLE,"Map(isManaged -> true, description -> by Legend-Delta from PURE entity [databricks::entity::employee], partitionBy -> [], properties -> {})",null,List(3591277798165548),0410-051949-9j5ul4vr,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


In [0]:
%scala
display(sql(s"SELECT * FROM $tableName VERSION AS OF ${versionLatest}"))

first_name,last_name,birth_date,gender,id,sme,joined_date,high_fives,isMvp
Levey,Storck,1989-02-19,M,null,C,2015-12-05,282,true
Maria,O'Gorman,1987-08-14,M,2,Python,2017-03-03,299,true
Evvy,Lepoidevin,1970-10-04,M,3,C,2020-11-02,182,true
Georges,Jotcham,1973-11-26,F,4,Scala,2020-09-14,229,true
Doroteya,Wadhams,1987-03-11,N,5,Scala,2019-02-11,78,true
Mia,Millgate,1988-08-01,F,6,Python,2017-04-13,146,true
Celene,Calverley,1979-07-15,N,7,Python,2021-06-03,69,true
Richie,Di Matteo,1980-05-18,F,8,Python,2014-08-23,167,true
Ignaz,Kurth,1987-01-10,F,null,Python,2014-02-01,199,true
Anthia,Duck,1998-02-08,F,10,Python,2015-01-14,277,true


In [0]:
%scala
display(sql(s"SELECT * FROM $tableName VERSION AS OF ${versionLatest - 1}"))

first_name,last_name,birth_date,gender,id,sme,joined_date,high_fives
Levey,Storck,1989-02-19,M,null,C,2015-12-05,282
Maria,O'Gorman,1987-08-14,M,2,Python,2017-03-03,299
Evvy,Lepoidevin,1970-10-04,M,3,C,2020-11-02,182
Georges,Jotcham,1973-11-26,F,4,Scala,2020-09-14,229
Doroteya,Wadhams,1987-03-11,N,5,Scala,2019-02-11,78
Mia,Millgate,1988-08-01,F,6,Python,2017-04-13,146
Celene,Calverley,1979-07-15,N,7,Python,2021-06-03,69
Richie,Di Matteo,1980-05-18,F,8,Python,2014-08-23,167
Ignaz,Kurth,1987-01-10,F,null,Python,2014-02-01,199
Anthia,Duck,1998-02-08,F,10,Python,2015-01-14,277


# Example - validate
With all of our expectations available from PURE to SQL, we can quickly validate quality of our delta table using the latest PURE constraints

In [0]:
%scala
import org.apache.spark.sql.functions._
val inputDF = spark.read.table("legend.employee")
val validationDF = inputDF.legendValidate(expectations, "legend").withColumn("legend", explode(col("legend")))
display(validationDF.limit(10))

first_name,last_name,birth_date,gender,id,sme,joined_date,high_fives,isMvp,legend
Levey,Storck,1989-02-19,M,null,C,2015-12-05,282,true,[sme] not allowed value
Evvy,Lepoidevin,1970-10-04,M,3,C,2020-11-02,182,true,[sme] not allowed value
Anthia,Duck,1998-02-08,F,10,Python,2015-01-14,277,true,[hiringAge] should be > 18
Ross,Bernhard,1988-12-25,F,17,C,2017-03-13,243,true,[sme] not allowed value
Eachelle,Martynka,1979-03-21,M,24,SAS,2021-01-27,156,true,[sme] not allowed value
Yanaton,Schultz,1999-04-16,F,27,Python,2016-07-30,261,true,[hiringAge] should be > 18
Ianthe,Sherwin,1998-12-17,M,30,Java,2014-07-21,26,true,[hiringAge] should be > 18
Bailie,Caltun,1979-11-19,F,31,C,2015-07-04,204,true,[sme] not allowed value
Rooney,Hacksby,1974-04-20,F,38,C,2015-04-12,134,true,[sme] not allowed value
Cecilius,Askham,1984-04-01,N,40,C,2014-11-20,36,true,[sme] not allowed value


# Example - read
From delta, we read objects that we transform back as a pure entity with derived properties and violated constraints. New derivations could be added from legend studio and seamlessly computed here without the need for engineering team to code. The generated dataframe would comply with business expectations and data quality, as defined from the legend studio.

In [0]:
%scala
val legendDf = legend.query("databricks::mapping::employee_delta")
display(legendDf.limit(10))

highFives,joinedDate,lastName,firstName,birthDate,id,sme,gender,hiringAge,age
282,2015-12-05,Storck,Levey,1989-02-19,null,C,M,26,33
299,2017-03-03,O'Gorman,Maria,1987-08-14,2,Python,M,30,35
182,2020-11-02,Lepoidevin,Evvy,1970-10-04,3,C,M,50,52
229,2020-09-14,Jotcham,Georges,1973-11-26,4,Scala,F,47,49
78,2019-02-11,Wadhams,Doroteya,1987-03-11,5,Scala,N,32,35
146,2017-04-13,Millgate,Mia,1988-08-01,6,Python,F,29,34
69,2021-06-03,Calverley,Celene,1979-07-15,7,Python,N,42,43
167,2014-08-23,Di Matteo,Richie,1980-05-18,8,Python,F,34,42
199,2014-02-01,Kurth,Ignaz,1987-01-10,null,Python,F,27,35
277,2015-01-14,Duck,Anthia,1998-02-08,10,Python,F,17,24


Given the following service defined on legend studio, we generate the corresponding spark execution plan and return a dataframe with all requested attributes and calculations

```
|databricks::entity::employee.all()->filter(
  x|$x.firstName->startsWith('G')
)->project(
  [
    x|$x.firstName,
    x|$x.lastName,
    x|$x.highFives,
    x|$x.age,
    x|$x.sme
  ],
  [
    'FirstName',
    'LastName',
    'HighFives',
    'Age',
    'Sme'
  ]
)->sort(
  [
    desc('HighFives')
  ]
)->take(10)
```

In [0]:
%scala
val legendDf = legend.query("databricks::service::employee")
display(legendDf.limit(10))

FirstName,LastName,HighFives,Age,Sme
Giustina,Pullen,300,45,Python
Giustina,Pullen,300,45,Python
Garth,Pucker,294,33,Python
Garth,Pucker,294,33,Python
Garv,Rulf,287,43,C
Garv,Rulf,287,43,C
Gonzales,Mewton,284,47,Python
Gonzales,Mewton,284,47,Python
Gib,Thorius,282,51,SAS
Gib,Thorius,282,51,SAS


The same works against aggregated functions like `groupBy`

```
|databricks::entity::employee.all()->filter(
  x|!($x.gender->isEmpty())
)->groupBy(
  [
    x|$x.gender
  ],
  [
    agg(
      x|$x.highFives,
      x|$x->average()
    ),
    agg(
      x|$x.id,
      x|$x->count()
    )
  ],
  [
    'Gender',
    'HighFives',
    'Employees'
  ]
)->sort(
  [
    desc('HighFives')
  ]
)->take(10)
```

In [0]:
%scala
val legendDf = legend.query("databricks::service::skills")
display(legendDf.limit(10))

Gender,HighFives,Employees
N,167.09091,88
M,152.09620,788
F,150.23941,1084
